<a href="https://colab.research.google.com/github/ameyas1/Alice_in_wonderland_text_generator/blob/master/Alice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
with open('alice.txt', 'r') as f:
  text = f.read()

In [3]:
text[:100]

'                     Down the Rabbit-Hole\n\n  Alice was beginning to get very tired of sitting by her'

In [0]:
from string import punctuation

text = text.lower()
text = ''.join([c for c in text if c not in punctuation])

In [0]:
words=text.split()

In [6]:
words[:100]

['down',
 'the',
 'rabbithole',
 'alice',
 'was',
 'beginning',
 'to',
 'get',
 'very',
 'tired',
 'of',
 'sitting',
 'by',
 'her',
 'sister',
 'on',
 'the',
 'bank',
 'and',
 'of',
 'having',
 'nothing',
 'to',
 'do',
 'once',
 'or',
 'twice',
 'she',
 'had',
 'peeped',
 'into',
 'the',
 'book',
 'her',
 'sister',
 'was',
 'reading',
 'but',
 'it',
 'had',
 'no',
 'pictures',
 'or',
 'conversations',
 'in',
 'it',
 'and',
 'what',
 'is',
 'the',
 'use',
 'of',
 'a',
 'book',
 'thought',
 'alice',
 'without',
 'pictures',
 'or',
 'conversation',
 'so',
 'she',
 'was',
 'considering',
 'in',
 'her',
 'own',
 'mind',
 'as',
 'well',
 'as',
 'she',
 'could',
 'for',
 'the',
 'hot',
 'day',
 'made',
 'her',
 'feel',
 'very',
 'sleepy',
 'and',
 'stupid',
 'whether',
 'the',
 'pleasure',
 'of',
 'making',
 'a',
 'daisychain',
 'would',
 'be',
 'worth',
 'the',
 'trouble',
 'of',
 'getting',
 'up',
 'and']

Unique words

In [7]:
len(set(words))

2733

total words

In [8]:
len(words)
    

26369

In [0]:
length=51
sequences=list()
for i in range(length,len(words)):
  seq=words[i-length:i]
  line=' '.join(seq)
  sequences.append(line)

In [10]:
print('total sequences:- %d'%(len(sequences)))

total sequences:- 26318


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(sequences)


In [0]:
line_sequence=tokenizer.texts_to_sequences(sequences)

In [13]:
line_sequence[:2]

[[37,
  1,
  1055,
  10,
  13,
  298,
  3,
  101,
  27,
  525,
  7,
  378,
  79,
  16,
  521,
  18,
  1,
  1467,
  2,
  7,
  377,
  130,
  3,
  52,
  138,
  56,
  679,
  5,
  21,
  1054,
  63,
  1,
  523,
  16,
  521,
  13,
  1053,
  22,
  6,
  21,
  44,
  822,
  56,
  2728,
  11,
  6,
  2,
  28,
  35,
  1,
  209],
 [1,
  1055,
  10,
  13,
  298,
  3,
  101,
  27,
  525,
  7,
  378,
  79,
  16,
  521,
  18,
  1,
  1467,
  2,
  7,
  377,
  130,
  3,
  52,
  138,
  56,
  679,
  5,
  21,
  1054,
  63,
  1,
  523,
  16,
  521,
  13,
  1053,
  22,
  6,
  21,
  44,
  822,
  56,
  2728,
  11,
  6,
  2,
  28,
  35,
  1,
  209,
  7]]

In [0]:
vocab_size=len(tokenizer.word_index)+1

In [15]:
vocab_size

2734

In [0]:
line_sequence=np.array(line_sequence)

In [0]:
x,y=line_sequence[:,:-1],line_sequence[:,-1]

In [0]:
from tensorflow.keras.utils import to_categorical

y=to_categorical(y,num_classes=vocab_size)

In [0]:
seq_len=x.shape[1]

In [20]:
seq_len

50

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import CuDNNLSTM,Embedding

In [0]:
model=Sequential()

In [23]:
model.add(Embedding(vocab_size,55,input_length =seq_len))

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
model.add(CuDNNLSTM(128,return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [25]:
model.add(Dense(vocab_size,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 55)            150370    
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (None, 50, 128)           94720     
_________________________________________________________________
dropout (Dropout)            (None, 50, 128)           0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               132096    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2734)              352686    
Total params: 729,872
Trainable params: 729,872
Non-trainable params: 0
_________________________________________________________________
None

In [26]:
model.fit(x,y, batch_size =128, nb_epoch = 500)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
26318/26318 [==============================] - 8s 302us/sample - loss: 6.3510 - acc: 0.0613
Epoch 2/500
26318/26318 [==============================] - 6s 246us/sample - loss: 6.0235 - acc: 0.0618
Epoch 3/500
26318/26318 [==============================] - 7s 247us/sample - loss: 5.9018 - acc: 0.0633
Epoch 4/500
26318/26318 [==============================] - 6s 246us/sample - loss: 5.7736 - acc: 0.0705
Epoch 5/500
26318/26318 [==============================] - 7s 248us/sample - loss: 5.6741 - acc: 0.0721
Epoch 6/500
26318/26318 [==============================] - 6s 247us/sample - loss: 5.5996 - acc: 0.0745
Epoch 7/500
26318/26318 [==============================] - 6s 245us/sample - loss: 5.5365 - acc: 0.0774
Epoch 8/500
26318/26318 [==============================] - 7s 247us/sample - loss: 5.4762 - acc: 0.0792
Epoch 9/500
26318/26318 [==============================] - 6s 246us/sample - loss: 5.4247 - acc: 0.0812
Epoch 10/500
263

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [33]:
text=sequences[6]
print(text+'\n')

to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought alice without



In [0]:
def generate(model,tokenizer,seq_length,org_text,n_words):
  result=list()
  text=org_text
  for _ in range(n_words):
    encoded=tokenizer.texts_to_sequences([text])[0]
    encoded=pad_sequences([encoded],maxlen=seq_length,truncating='pre')
    y=model.predict_classes(encoded,verbose=0)
    out_word=''
    for word,index in tokenizer.word_index.items():
      if index==y:
        out_word=word
        break
    text +=' '+word
    result.append(out_word)
  return ' '.join(result)


In [0]:
generated=generate(model,tokenizer,seq_len,text,100)

In [37]:
print(generated)

pictures or conversation so she was considering in her own mind as well as she could for the hot day said the king and she went on again yelled the gryphon had a friend of minea cheshire cat said alice allow me to introduce it i dont like it off why i should like them i eat it and said the cat and vanished for the other side of the garden where i didnt know it said the king and the hatter was to look in a chrysalisyou will some day you knowand all over his headbrandy nowdont choke himhow
